<a href="https://colab.research.google.com/github/hanansuk/guns_n_roses/blob/main/impute_cdc_monthly_state_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd

In [81]:
token = 'ADD TOKEN HERE'
suppressed = pd.read_csv(f'https://raw.githubusercontent.com/hanansuk/guns_n_roses/main/cdc_by_state_month_with_suppressed_values.csv?token={token}')

HTTPError: ignored

In [44]:
token = 'ADD TOKEN HERE'
aggregate = pd.read_csv(f'https://raw.githubusercontent.com/hanansuk/guns_n_roses/main/cdc_by_state.csv?token={token}')

In [46]:
suppressed.head()

,State,State.Code,Year,Year.Code,Month,Month.Code,Deaths
0,Alabama,1,2021,2021,"Jan., 2021",2021/01,122
1,Alabama,1,2021,2021,"Feb., 2021",2021/02,104
2,Alabama,1,2021,2021,"Mar., 2021",2021/03,103
3,Alabama,1,2021,2021,"Apr., 2021",2021/04,102
4,Alabama,1,2021,2021,"May, 2021",2021/05,108


In [47]:
aggregate.head()

,State,State.Code,Year,Year.Code,Deaths,Population,Crude.Rate,YoY
0,Alabama,1,1999,1999,787,4430141,17.8,NaN
1,Alabama,1,2000,2000,764,4447100,17.2,-0.030105
2,Alabama,1,2001,2001,737,4467634,16.5,-0.036635
3,Alabama,1,2002,2002,724,4480089,16.2,-0.017956
4,Alabama,1,2003,2003,764,4503491,17.0,0.052356


In [91]:
combined = suppressed.merge(aggregate, on=['State.Code', 'Year'])
combined.rename({
    'State_x': 'state',
    'Year': 'year',
    'Month.Code': 'period',
    'Deaths_x': 'monthly_gun_deaths',
    'Deaths_y': 'yearly_gun_deaths'
}, axis=1, inplace=True)
combined = combined[['state', 'year', 'period', 'monthly_gun_deaths', 'yearly_gun_deaths']]
combined['monthly_gun_deaths'] = [int(x) if x != 'Suppressed' else x for x in combined['monthly_gun_deaths']]
combined['yearly_gun_deaths'] = combined['yearly_gun_deaths'].astype('int')

In [92]:
missing_values_per_year = (
    combined[combined['monthly_gun_deaths'] == 'Suppressed']
    .groupby(['state', 'year'])
    .count()
    .period
    .reset_index())
combined = combined.merge(missing_values_per_year, on=['state', 'year'])
combined.rename({'period_y': 'missing_values_per_year', 'period_x': 'period'}, axis=1, inplace=True)

In [93]:
total_monthly_not_missing = (
    combined[combined['monthly_gun_deaths'] != 'Suppressed']
    .groupby(['state', 'year'])
    .sum(numeric_only=False)
    .monthly_gun_deaths
    .reset_index())
combined = combined.merge(total_monthly_not_missing, on=['state', 'year'])
combined.rename({'monthly_gun_deaths_y': 'total_monthly_not_missing', 'monthly_gun_deaths_x': 'monthly_gun_deaths'}, axis=1, inplace=True)

In [95]:
combined['imputed_monthly'] = (combined.yearly_gun_deaths - combined.total_monthly_not_missing) / combined.missing_values_per_year
combined['monthly_gun_deaths'] = [x.imputed_monthly if x.monthly_gun_deaths == 'Suppressed' else x.monthly_gun_deaths for _, x in combined.iterrows()]

In [97]:
res = combined[['state', 'year', 'period', 'monthly_gun_deaths']]

In [98]:
from google.colab import files
res.to_csv('cdc_monthly_state_gun_deaths_imputed.csv', encoding = 'utf-8-sig')
files.download('cdc_monthly_state_gun_deaths_imputed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>